# Libs

In [1]:
import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

import os
import glob
import lasio
import dlisio as dlis

pd.options.display.max_columns = 10
pd.options.display.precision = 5
pd.options.display.max_colwidth = 30
pd.options.display.max_rows = 6

# Search of las & dlis files

In [2]:
# Search in which directories exist las/dlis files
root_folder = r'C:\jupyter\proj_volve_geodata_analytics\Well_logs_pr_WELL'
las_files = []

for dirpath, dirnames, filenames in os.walk(root_folder):
    for filename in filenames:
        if filename.endswith('.las'):
            las_files.append(os.path.join(dirpath, filename))
        if filename.endswith('.LAS'):
            las_files.append(os.path.join(dirpath, filename))
        if filename.endswith('.dlis'):
            las_files.append(os.path.join(dirpath, filename))
        if filename.endswith('.DLIS'):
            las_files.append(os.path.join(dirpath, filename))

df_lst = []
for las_file in las_files:
    df_files = pd.DataFrame((las_file.split('\\')[4:])).T
    df_lst.append(df_files)
df_files = pd.concat(df_lst).reset_index(drop=True)
df_files.columns = ['well','folder','path_1', 'path_2']
df_files.sort_values(by='well', inplace=True)
df_files.folder.unique()

array(['05.PETROPHYSICAL INTERPRETATION', '06.LFP', '13.GEOCHEM',
       '04.COMPOSITE', '03.PRESSURE', '02.LWD_EWL', '01.MUD_LOG',
       '10.PRODUCTION LOGS', '11. INTEGRITY LOGS', '07.IMAGE',
       '08.VSP_VELOCITY', '12.BIOSTRAT'], dtype=object)

In [4]:
cpi_files

,well,folder,path_1,path_2
0,15_9-19 A,05.PETROPHYSICAL INTERPRET...,CPI,15_9-19_A_CPI.las
1,15_9-19 B&BT2,05.PETROPHYSICAL INTERPRET...,CPI,15_9-19_BT2_CPI.las
2,15_9-19 S&SR,05.PETROPHYSICAL INTERPRET...,CPI,15_9-19_SR_CPI.las


In [5]:
interp_files

,well,folder,path_1,path_2
0,15_9-F-1,05.PETROPHYSICAL INTERPRET...,WLC_PETRO_COMPUTED_OUTPUT_...,NaN
1,15_9-F-1,05.PETROPHYSICAL INTERPRET...,WLC_PETRO_COMPUTED_INPUT_1...,NaN
2,15_9-F-1 A,05.PETROPHYSICAL INTERPRET...,WLC_PETRO_COMPUTED_INPUT_1...,NaN
...,...,...,...,...
32,15_9-F-4,05.PETROPHYSICAL INTERPRET...,WLC_PETRO_COMPUTED_OUTPUT_...,NaN
33,15_9-F-5,05.PETROPHYSICAL INTERPRET...,WLC_PETRO_COMPUTED_OUTPUT_...,NaN
34,15_9-F-5,05.PETROPHYSICAL INTERPRET...,WLC_PETRO_COMPUTED_INPUT_1...,NaN


In [3]:
# Folders with las files in CPI folder and in PETROPHYSICAL INTERPRETATION folder
cpi_files = df_files[(df_files.folder == '05.PETROPHYSICAL INTERPRETATION') & 
                     (df_files.path_1 == 'CPI')].reset_index(drop=True)
interp_files = df_files[(df_files.folder == '05.PETROPHYSICAL INTERPRETATION') & 
                        (df_files.path_1 != 'CPI') & (df_files.path_2.isna())].reset_index(drop=True)
pp_files_wells = np.hstack((cpi_files.well.unique(), interp_files.well.unique()))
interp_files

,well,folder,path_1,path_2
0,15_9-F-1,05.PETROPHYSICAL INTERPRET...,WLC_PETRO_COMPUTED_OUTPUT_...,NaN
1,15_9-F-1,05.PETROPHYSICAL INTERPRET...,WLC_PETRO_COMPUTED_INPUT_1...,NaN
2,15_9-F-1 A,05.PETROPHYSICAL INTERPRET...,WLC_PETRO_COMPUTED_INPUT_1...,NaN
...,...,...,...,...
32,15_9-F-4,05.PETROPHYSICAL INTERPRET...,WLC_PETRO_COMPUTED_OUTPUT_...,NaN
33,15_9-F-5,05.PETROPHYSICAL INTERPRET...,WLC_PETRO_COMPUTED_OUTPUT_...,NaN
34,15_9-F-5,05.PETROPHYSICAL INTERPRET...,WLC_PETRO_COMPUTED_INPUT_1...,NaN


In [4]:
# Folders with dlis files
one_more_files = df_files[~df_files.well.isin(pp_files_wells)].reset_index(drop=True)
display(one_more_files.well.unique())
one_more_files

array(['15_9-F-7', '15_9-F-9', '15_9-F-9 A'], dtype=object)

,well,folder,path_1,path_2
0,15_9-F-7,02.LWD_EWL,WL_RAW_GR-REMP_MWD_1.DLIS,NaN
1,15_9-F-7,04.COMPOSITE,WLC_PETROPHYSICAL_COMPOSIT...,NaN
2,15_9-F-7,02.LWD_EWL,WL_RAW_GR_MWD_1.DLIS,NaN
...,...,...,...,...
20,15_9-F-9 A,02.LWD_EWL,WL_RAW_GR-REMP_MWD_1.DLIS,NaN
21,15_9-F-9 A,02.LWD_EWL,WL_RAW_GR_MWD_1.DLIS,NaN
22,15_9-F-9 A,04.COMPOSITE,WLC_PETROPHYSICAL_COMPOSIT...,NaN


In [5]:
# Displaying dlis files related to petrophysic 
one_more_files[one_more_files.path_1.str.contains('PETROPHYSICAL_COMPOSITE')]

,well,folder,path_1,path_2
1,15_9-F-7,04.COMPOSITE,WLC_PETROPHYSICAL_COMPOSIT...,NaN
9,15_9-F-9,04.COMPOSITE,WLC_PETROPHYSICAL_COMPOSIT...,NaN
22,15_9-F-9 A,04.COMPOSITE,WLC_PETROPHYSICAL_COMPOSIT...,NaN


# Reading las-files for Exploration (CPI) wells

In [13]:
# Las files for Exploration wells probable with PP evaluation without input wireline logs
las_data = []
for i in range(len(cpi_files)):
    full_path = os.path.join(root_folder, cpi_files.well[i], cpi_files.folder[i], cpi_files.path_1[i], cpi_files.path_2[i])
    las = lasio.read(full_path)
    df = las.df()
    df['WELL'] = cpi_files.well[i]
    las_data.append(df)
cpi_data = pd.concat(las_data).reset_index()
cpi_data = cpi_data[['WELL', 'DEPTH', 'BS', 'BVW', 'CARB_FLAG', 'COAL_FLAG', 'DELTA', 'GRMAX',
                    'GRMIN', 'J', 'KLHC_INT', 'KLOGH', 'M', 'N', 'PERF_FLAG', 'PHIF',
                    'PORD', 'RHOFL', 'RHOMA', 'RW', 'SAND_FLAG', 'SW', 'SWIRR', 'TEMP',
                    'VSH',  'KLOGV', 'BWV', 'DT']]
cpi_data.columns = cpi_data.columns.str.lower()
cpi_data['well'] = cpi_data['well'].astype('string')
cpi_data['depth'] = cpi_data['depth'].round(1)
# Save to parquet file to save disk space
cpi_data.to_parquet('Project/input/cpi_data.parquet', index=False)

Only engine='normal' can read wrapped files


# Las-files with input & output data

In [7]:
# Making up df with input and output pathes
interp_files_input = []
interp_files_output = []
for idx, row in interp_files.iterrows():
    if 'INPUT' in row['path_1']:
        interp_files_input.append(interp_files.iloc[idx])
    if 'OUTPUT' in row['path_1']:
        interp_files_output.append(interp_files.iloc[idx])
df_interp_files_input = pd.DataFrame(interp_files_input).reset_index(drop=True)
df_interp_files_output = pd.DataFrame(interp_files_output).reset_index(drop=True)

In [62]:
# Reading las files for prod/inject wells
las_data = []
las_file_err = []
for i in tqdm(range(len(interp_files_input))):
    full_path = os.path.join(root_folder,   df_interp_files_input.well[i], 
                                            df_interp_files_input.folder[i], 
                                            df_interp_files_input.path_1[i])
    if 'las' in full_path or 'LAS' in full_path:
        las = lasio.read(full_path)
        df = las.df()
        df['WELL'] = df_interp_files_input.well[i]
        las_data.append(df)
    else:
        las_file_err.append(full_path)
interp_logs_input = pd.concat(las_data).reset_index()
# Rearrange columns to proper order
interp_logs_input = interp_logs_input[[  'WELL', 'DEPTH', 'BS', 'CALI', 'GR',  
                                         'RHOB', 'DRHO', 'PEF', 'NPHI', 'RT', 'DT', 'DTS',  'RS', 'RM', 'RD', 
                                         'ROP', 'ROP5_RM', 'CARB_FLAG', 'COAL_FLAG', 'SAND_FLAG',   
                                         'ABDCQF01', 'ABDCQF02', 'ABDCQF03', 'ABDCQF04',
                                         'NBGRCFM','RACEHM', 'RACELM', 'RPCEHM', 'RPCELM']]
# Data type processing
interp_logs_input.columns = interp_logs_input.columns.str.lower()
interp_logs_input['well'] = interp_logs_input['well'].astype('string')
interp_logs_input['depth'] = interp_logs_input['depth'].round(1)
# Save to parquet file to save disk space
interp_logs_input.to_parquet('Project/input/interp_logs_input.parquet', index=False)

100%|██████████| 18/18 [00:04<00:00,  4.03it/s]


In [64]:
las_data = []
las_file_err = []
for i in tqdm(range(len(df_interp_files_output))):
    full_path = os.path.join(root_folder,   df_interp_files_output.well[i], 
                                            df_interp_files_output.folder[i], 
                                            df_interp_files_output.path_1[i])
    if 'las' in full_path or 'LAS' in full_path:
        las = lasio.read(full_path)
        df = las.df()
        df['WELL'] = df_interp_files_input.well[i]
        las_data.append(df)
    else:
        las_file_err.append(full_path)
interp_logs_output = pd.concat(las_data).reset_index()
# Rearrange columns to proper order
interp_logs_output = interp_logs_output[['WELL', 'DEPTH', 'BADDATA_FLAG', 'SAND_FLAG', 'VSH', 'PHIF', 'KLOGH', 'SW', 'BVW','VCARB', 'CARB_FLAG', 'COAL_FLAG']]
# Data type processing
interp_logs_output.columns = interp_logs_output.columns.str.lower()
interp_logs_output['well'] = interp_logs_output['well'].astype('string')
interp_logs_output['depth'] = interp_logs_output['depth'].round(1)
# Save to parquet file to save disk space
interp_logs_output.to_parquet('Project/input/interp_logs_output.parquet', index=False)

100%|██████████| 17/17 [00:00<00:00, 30.68it/s]


In [58]:
# Revision of presence or absence wells in interp_logs_input
for well in interp_logs_input.well.unique():
    if well in interp_logs_output.well.unique():
        pass
    else:
        print(f'{well} - no pp_eval file')

15_9-F-12 - no pp_eval file
15_9-F-15 D - no pp_eval file


In [59]:
# Revision of presence or absence wells in interp_logs_input
for well in interp_logs_output.well.unique():
    if well in interp_logs_input.well.unique():
        pass
    else:
        print(f'{well} - no input log file')

15_9-F-15 C - no input log file


# Comment about DLIS files

In [ ]:
# DLIS files will be precessend in separate ipynb file
dlis_wells_lst = []
for str_wells in las_file_err:
    dlis_wells_lst.append(str_wells.split('\\')[4])
print(dlis_wells_lst)